In [ ]:
import scanpy as sc
import scvi
from tqdm.notebook import tqdm
import os
import numpy as np
import pandas as pd
from scvi.model.utils import mde
import matplotlib.pyplot as plt
import pickle
import glob

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import torch

In [2]:
sc.set_figure_params(figsize=(4, 4))

Add the paths to all folders for experiments to be integrated

In [3]:
data_dir = "/mnt/sata1/Analysis_Alex/uninfected"
input_folders = glob.glob(os.path.join(data_dir, "segmentation_S*"))

In [4]:
output_folder = os.path.join(data_dir, 'analysis', 'cleaned')

In [5]:
try:
    os.makedirs(output_folder)
except FileExistsError:
    pass

Map to reference

In [ ]:
reference_adata = (
    r"timecourse.h5ad"
)

reference_adata = sc.read(reference_adata)

In [ ]:
scvi.model.SCVI.setup_anndata(reference_adata, batch_key="batch", layer="raw")

In [9]:
torch.set_float32_matmul_precision('high')

In [ ]:
scvi_ref = scvi.model.SCVI(reference_adata, n_layers=2, n_latent=30, gene_likelihood="nb")
scvi_ref.train()

In [11]:
SCVI_LATENT_KEY = "X_scVI_replicates"

reference_adata.obsm[SCVI_LATENT_KEY] = scvi_ref.get_latent_representation()
sc.pp.neighbors(reference_adata, use_rep=SCVI_LATENT_KEY)

In [ ]:
sc.tl.umap(reference_adata)
sc.pl.umap(reference_adata, color = 'Subtype')

Save the mapping projection to the adata

In [ ]:

for t in range(len(input_folders)):
    target_adata = sc.read(os.path.join(input_folders[t], 'adatas', '04_tissue_cleared.h5ad'))
    target_adata.obs['batch'] = os.path.basename(input_folders[t])
    scvi.model.SCVI.prepare_query_anndata(target_adata, scvi_ref)
    scvi_query = scvi.model.SCVI.load_query_data(
        target_adata,
        scvi_ref
    )

    scvi_query.train(max_epochs=200, plan_kwargs={"weight_decay": 0.0})
    target_adata.obsm[SCVI_LATENT_KEY] = scvi_query.get_latent_representation()
    target_adata.write(os.path.join(input_folders[t], 'adatas', '06_reference_mapped.h5ad'))